In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1260
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [5]:
function kalman_filter_graph()
    x_prev_add = AdditionNode()
    
    x_prior = datavar(:x_prior, Normal{Float64})
    add_1   = constvar(:add_1, 1.0)

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    
    add_x_and_noise = AdditionNode()
    
    x = simplerandomvar(:x)
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = datavar(:y, Float64)
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(x_prev_add)
    activate!(add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()

    link(x_prior, getbelief(x))
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    beliefs = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getbelief(x) |> enumerate(), (t) -> beliefs[t[1]] = getdata(t[2]))
    
    for (i, d) in enumerate(data)
        update!(y, d)
    end
    
    return beliefs
end

kalman (generic function with 1 method)

In [6]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [9]:
@btime kalman($data)

  963.022 μs (16107 allocations: 695.03 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-10.608472166455199, σ=14.002800840280099)
 Normal{Float64}(μ=-0.38153395640294574, σ=9.950371902099892)
 Normal{Float64}(μ=3.6735959293698133, σ=8.137884587711596)
 Normal{Float64}(μ=6.671384853647265, σ=7.053456158585984)
 Normal{Float64}(μ=10.04991620683911, σ=6.311944030978033)
 Normal{Float64}(μ=11.75568265580858, σ=5.763904177042351)
 Normal{Float64}(μ=10.15299554322442, σ=5.337605126836238)
 Normal{Float64}(μ=12.470812863691538, σ=4.993761694389224)
 Normal{Float64}(μ=15.230572959758403, σ=4.708816093480111)
 Normal{Float64}(μ=17.72075844792929, σ=4.467670516087703)
 Normal{Float64}(μ=19.743968217110282, σ=4.2601432284230505)
 Normal{Float64}(μ=21.991182360441773, σ=4.079085082240021)
 Normal{Float64}(μ=23.375046018060587, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=589.8639396813974, σ=0.582706653680032)
 Normal{Float64}(μ=590.8719733176546, σ=0.5822126418123303)
 Normal{Float64}(μ=591.8368577415619, σ=0.5817198842703781)


In [8]:
@time kalman(data)

  0.001563 seconds (16.11 k allocations: 695.031 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-10.608472166455199, σ=14.002800840280099)
 Normal{Float64}(μ=-0.38153395640294574, σ=9.950371902099892)
 Normal{Float64}(μ=3.6735959293698133, σ=8.137884587711596)
 Normal{Float64}(μ=6.671384853647265, σ=7.053456158585984)
 Normal{Float64}(μ=10.04991620683911, σ=6.311944030978033)
 Normal{Float64}(μ=11.75568265580858, σ=5.763904177042351)
 Normal{Float64}(μ=10.15299554322442, σ=5.337605126836238)
 Normal{Float64}(μ=12.470812863691538, σ=4.993761694389224)
 Normal{Float64}(μ=15.230572959758403, σ=4.708816093480111)
 Normal{Float64}(μ=17.72075844792929, σ=4.467670516087703)
 Normal{Float64}(μ=19.743968217110282, σ=4.2601432284230505)
 Normal{Float64}(μ=21.991182360441773, σ=4.079085082240021)
 Normal{Float64}(μ=23.375046018060587, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=589.8639396813974, σ=0.582706653680032)
 Normal{Float64}(μ=590.8719733176546, σ=0.5822126418123303)
 Normal{Float64}(μ=591.8368577415619, σ=0.5817198842703781)


In [7]:
@time kalman(data)

  2.531156 seconds (7.00 M allocations: 373.271 MiB, 2.93% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-10.608472166455199, σ=14.002800840280099)
 Normal{Float64}(μ=-0.38153395640294574, σ=9.950371902099892)
 Normal{Float64}(μ=3.6735959293698133, σ=8.137884587711596)
 Normal{Float64}(μ=6.671384853647265, σ=7.053456158585984)
 Normal{Float64}(μ=10.04991620683911, σ=6.311944030978033)
 Normal{Float64}(μ=11.75568265580858, σ=5.763904177042351)
 Normal{Float64}(μ=10.15299554322442, σ=5.337605126836238)
 Normal{Float64}(μ=12.470812863691538, σ=4.993761694389224)
 Normal{Float64}(μ=15.230572959758403, σ=4.708816093480111)
 Normal{Float64}(μ=17.72075844792929, σ=4.467670516087703)
 Normal{Float64}(μ=19.743968217110282, σ=4.2601432284230505)
 Normal{Float64}(μ=21.991182360441773, σ=4.079085082240021)
 Normal{Float64}(μ=23.375046018060587, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=589.8639396813974, σ=0.582706653680032)
 Normal{Float64}(μ=590.8719733176546, σ=0.5822126418123303)
 Normal{Float64}(μ=591.8368577415619, σ=0.5817198842703781)
